In [1]:
import ROOT
import numpy as np
from itertools import combinations, combinations_with_replacement, product

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x6de9990


In [2]:
%jsroot on

In [3]:
# FIXME: don't touch! the fortran code has a global state :(
# ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
# plot_path = "plots/angles/mg5_1M"
plot_path = "/dev/null"
oo_plot_name = "OO_wz.pdf"

In [5]:
ROOT.gSystem.Load("OO/diehl/libObservable.so")

0

In [6]:
ROOT.gInterpreter.Declare("#include <WWTools.h>")
ROOT.gInterpreter.Declare("#include \"OO/diehl/OOTools.h\"")
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")

True

In [7]:
# use same masses as in whizard!
mW = 80.419
mZ = 91.1882
# or use Markus old values, whatever the file was generated with obviously
# mW = 80.22
# mZ = 91.17
# sqrt_s = 250
# sqrt_s = 240
sqrt_s = 190
ROOT.OOTools.init(sqrt_s, mW, mZ)

s = sqrt_s**2
cw2 = (mW/mZ)**2
sw2 = 1 - cw2

In [8]:
ROOT.ildStyle.SetOptStat(1)
# ROOT.ildStyle.SetOptStat(0)

In [9]:
%%cpp
using namespace ROOT::VecOps;

In [10]:
# df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_190/ww_ac_e-1.edm4hep.root")
df = ROOT.RDataFrame("events", "data/gen/whizard_sw_forced_ww_semi_190/ww.edm4hep.root")
# df = ROOT.RDataFrame("events", "data/gen/whizard_ww_semi_190_polarized/ww.edm4hep.root")
# n_events = 50000
# n_events = 100000
n_events = 1000000
df = df.Range(n_events)

In [11]:

df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")
df = df.Define("weight", "(double) Parameters.get<float>(\"_weight\").value()")
df = df.Define("weight_alt1", "Parameters.get<double>(\"weight_alt1\").value()")
df = df.Define("weight_alt2", "Parameters.get<double>(\"weight_alt2\").value()")
df = df.Define("weight_alt3", "Parameters.get<double>(\"weight_alt3\").value()")
df = df.Define("weight_alt4", "Parameters.get<double>(\"weight_alt4\").value()")
df = df.Define("weight_alt5", "Parameters.get<double>(\"weight_alt5\").value()")
df = df.Define("weight_alt6", "Parameters.get<double>(\"weight_alt6\").value()")
# FIXME: debug only
# df = df.Define("weight_alt1", "1.0")
# df = df.Define("weight_alt2", "1.0")
# df = df.Define("weight_alt3", "1.0")

In [12]:
df = df.Vary("weight", "RVec<double>{weight * weight_alt1, weight * weight_alt2, weight * weight_alt3}", ["g1z_up", "ka_up", "la_up"], "aTGC")

In [13]:
# these files are very simple, they have the Ws explicitly and there is only one stable e/p
df = df.Define("e_MC_idx", "ArgMax(MCParticles.PDG == 11 && MCParticles.generatorStatus == 1)")
df = df.Define("enu_MC_idx", "ArgMax(MCParticles.PDG == -12 && MCParticles.generatorStatus == 1)")
# only named p and pnu out of historical reasons, in reality these are the quarks
df = df.Define("p_MC_idx", "ArgMax(MCParticles.PDG == -1 && MCParticles.generatorStatus == 1)")
df = df.Define("pnu_MC_idx", "ArgMax(MCParticles.PDG == 2 && MCParticles.generatorStatus == 1)")
df = df.Define("Wp_MC_idx", "ArgMax(MCParticles.PDG == 24)")
df = df.Define("Wm_MC_idx", "ArgMax(MCParticles.PDG == -24)")

df = df.Define("e_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[e_MC_idx], MCParticles.momentum.y[e_MC_idx], MCParticles.momentum.z[e_MC_idx], MCParticles.mass[e_MC_idx])")
df = df.Define("enu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[enu_MC_idx], MCParticles.momentum.y[enu_MC_idx], MCParticles.momentum.z[enu_MC_idx], MCParticles.mass[enu_MC_idx])")
df = df.Define("p_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[p_MC_idx], MCParticles.momentum.y[p_MC_idx], MCParticles.momentum.z[p_MC_idx], MCParticles.mass[p_MC_idx])")
df = df.Define("pnu_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[pnu_MC_idx], MCParticles.momentum.y[pnu_MC_idx], MCParticles.momentum.z[pnu_MC_idx], MCParticles.mass[pnu_MC_idx])")
df = df.Define("Wp_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wp_MC_idx], MCParticles.momentum.y[Wp_MC_idx], MCParticles.momentum.z[Wp_MC_idx], MCParticles.mass[Wp_MC_idx])")
df = df.Define("Wm_MC_lvec", "ROOT::Math::PxPyPzMVector(MCParticles.momentum.x[Wm_MC_idx], MCParticles.momentum.y[Wm_MC_idx], MCParticles.momentum.z[Wm_MC_idx], MCParticles.mass[Wm_MC_idx])")

# XXX: comment in for sw files
df = df.Redefine("Wp_MC_lvec", "p_MC_lvec + pnu_MC_lvec")
df = df.Redefine("Wm_MC_lvec", "e_MC_lvec + enu_MC_lvec")

In [14]:
df = df.Define("W_minus_lvec", "Wm_MC_lvec")
df = df.Define("W_plus_lvec", "Wp_MC_lvec")
df = df.Define("e_minus_vec", "ROOT::Math::XYZVector(0, 0, 1)")
df = df.Define("e_plus_vec", "ROOT::Math::XYZVector(0, 0, -1)")

df = df.Define("CosTheta_W_minus", "ROOT::Math::VectorUtil::CosTheta(W_minus_lvec, e_minus_vec)")

# df = df.Define("CosTheta_W_plus", "ROOT::Math::VectorUtil::CosTheta(W_plus_lvec, e_minus_vec)")

In [15]:
df = df.Define("enu_m", "(e_MC_lvec + enu_MC_lvec).M()")
df = df.Define("pnu_m", "(p_MC_lvec + pnu_MC_lvec).M()")

In [16]:
df = df.Define("e_transformed_lvec", "WWTools::starVectorHagiwara(W_minus_lvec, e_MC_lvec, {0, 0, 1}, W_minus_lvec)")
df = df.Define("p_transformed_lvec", "WWTools::starVectorHagiwara(W_plus_lvec, p_MC_lvec, {0, 0, 1}, W_minus_lvec)")

In [17]:

df = df.Define("e_theta_prime", "e_transformed_lvec.Theta()")
df = df.Define("e_phi_prime", "e_transformed_lvec.Phi()")

df = df.Define("p_theta_prime", "p_transformed_lvec.Theta()")
df = df.Define("p_phi_prime", "p_transformed_lvec.Phi()")

df = df.Define("e_cosTheta_prime", "cos(e_theta_prime)")
df = df.Define("p_cosTheta_prime", "cos(p_theta_prime)")

In [18]:
df = df.Define("co", "CosTheta_W_minus")
df = df.Define("co1", "e_cosTheta_prime")
df = df.Define("co2", "p_cosTheta_prime")
df = df.Define("ph1", "e_phi_prime")
df = df.Define("ph2", "p_phi_prime")

In [19]:
df = df.Define("S_0", "OOTools::proba(co, co1, co2, ph1, ph2, 0, false, 0)")
df = df.Define("S_1_1", "OOTools::proba(co, co1, co2, ph1, ph2, 1, false, 1)")
df = df.Define("S_1_2", "OOTools::proba(co, co1, co2, ph1, ph2, 2, false, 1)")
df = df.Define("S_1_3", "OOTools::proba(co, co1, co2, ph1, ph2, 3, false, 1)")

In [20]:
df = df.Define("O_1", "S_1_1 / S_0")
df = df.Define("O_2", "S_1_2 / S_0")
df = df.Define("O_3", "S_1_3 / S_0")
# for i, j in combinations_with_replacement((1, 2, 3), r=2):
# for i, j in product((1, 2, 3), repeat=2):
#     df = df.Define(f"O_{i}O_{j}", f"O{i} * O{j}")

In [21]:
alt_setup = 1
for j in range(1, 4):
    df = df.Define(f"a_{j}", f"O_{j} * weight_alt{alt_setup}")
    df = df.Define(f"b_{j}", f"O_{j}")
    for i in range(1, 4):
        df = df.Define(f"c_{i}{j}", f"O_{i} * O_{j}")

In [22]:
# FIXME: for debugging
# df = df.Define("weight", "1")
# df = df.Filter("l_MC_lvec_pt > 2", "pt cut")
# df = df.Filter("abs(l_MC_lvec_cosTheta) < 0.975", "cosTheta cut") # ~6.5 deg
df = df.Filter(f"abs(enu_m - {mW}) < 2.5 && abs(pnu_m - {mW}) < 2.5")
# df = df.Filter(f"abs(enu_m - {mW}) < 1.25 && abs(pnu_m - {mW}) < 1.25")

In [23]:
m_a = []
m_b = []
m_c = []

for j in range(1, 4):
    # m_a.append(df.Mean(f"a_{j}"))
    # m_b.append(df.Mean(f"b_{j}"))
    m_a.append(df.Sum(f"a_{j}"))
    m_b.append(df.Sum(f"b_{j}"))
    for i in range(1, 4):
        # m_c.append(df.Mean(f"c_{i}{j}"))
        m_c.append(df.Sum(f"c_{i}{j}"))

In [24]:
h_enu_m = df.Histo1D("enu_m")
h_pnu_m = df.Histo1D("pnu_m")

In [25]:
h_CosTheta_W_minus = df.Histo1D(("", ";cos #theta_{W^{-}}", 40, -1., 1.), "CosTheta_W_minus", "weight")
# h_CosTheta_W_plus = df.Histo1D(("", ";cos #theta_{W^{+}}", 40, -1., 1.), "CosTheta_W_plus", "weight")
# h_l_theta_prime = df.Histo1D("l_theta_prime", "weight")
h_l_plus_theta_prime = df.Histo1D("p_theta_prime", "weight")
h_l_plus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{+}}", 40, -1., 1.), "p_cosTheta_prime", "weight")
h_l_plus_phi_prime = df.Histo1D(("", ";#phi*_{e+}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "p_phi_prime", "weight")
h_l_minus_theta_prime = df.Histo1D("e_theta_prime", "weight")
h_l_minus_cosTheta_prime = df.Histo1D(("", ";cos #theta*_{e^{-}}", 40, -1., 1.), "e_cosTheta_prime", "weight")
h_l_minus_phi_prime = df.Histo1D(("", ";#phi*_{e-}", 40, -ROOT.Math.Pi(), ROOT.Math.Pi()), "e_phi_prime", "weight")

In [30]:
a = np.asarray([m.GetValue() for m in m_a])
b = np.asarray([m.GetValue() for m in m_b])
c = np.asarray([m.GetValue() for m in m_c]).reshape((3, 3))
print(a)
print(b)
print(c)
g = np.linalg.solve(c, a-b)
print(g)
# TODO: convert to LEP parametrisation, use vary to see all 3 results?
d = s / (2*mW**2)
# convert Hagiwara delta f_i^Z to delta g_1^Z, delta kappa_Z, lambda_Z
fz2LEPZ = np.asarray([
    [1.,    -d,     0.],
    [-1.,   d-1.,   1.],
    [0.,    1.,     0.]
])
# convert delta kappa_Z to delta kappa_gamma
e = cw2 / sw2
LEPZ2LEP =  np.asarray([
    [1., 0., 0.],
    [e,  -e, 0.],
    [0., 0., 1.]
])
fz2LEP = LEPZ2LEP @ fz2LEPZ
print(fz2LEPZ @ g)
print(f"res: {fz2LEP @ g}")
# errors
V_gamma = np.linalg.inv(c)
# fz errors
print(np.sqrt(V_gamma.diagonal()))
# LEP errors?
V_gamma_LEP = fz2LEP @ V_gamma @ fz2LEP.T
print(V_gamma_LEP)
print(np.sqrt(V_gamma_LEP.diagonal()))

def normalize(matrix):
# attempt to convert to a correlation matrix...
    res = np.zeros_like(matrix)
    for i, j in product((0, 1, 2), repeat=2):
        res[i, j] = matrix[i, j] / np.sqrt(matrix[i, i] * matrix[j,j])
    return res


print(normalize(V_gamma))
print(normalize(V_gamma_LEP))

[4969.59758315  694.43524891 7683.35447338]
[  8076.64217576  -6671.46676118 -13043.05596019]
[[ 36009.32250433 -26385.11456607 -33623.85382136]
 [-26385.11456607  34102.3508052   49468.44679513]
 [-33623.85382136  49468.44679513 118170.69328112]]
[ 0.10482521 -0.00150872  0.20585199]
[ 0.10903604  0.09832466 -0.00150872]
res: [ 0.10903604  0.03748382 -0.00150872]
[0.00813004 0.01142316 0.00471269]
[[ 0.00074348  0.00380008 -0.00030345]
 [ 0.00380008  0.01955843 -0.00153649]
 [-0.00030345 -0.00153649  0.00013049]]
[0.02726679 0.13985146 0.01142316]
[[ 1.          0.65409333 -0.17284174]
 [ 0.65409333  1.         -0.69362479]
 [-0.17284174 -0.69362479  1.        ]]
[[ 1.          0.99653439 -0.97423459]
 [ 0.99653439  1.         -0.96177859]
 [-0.97423459 -0.96177859  1.        ]]


In [27]:
c_enu_m = ROOT.TCanvas()
h_enu_m.Draw()
c_enu_m.Draw()

c_pnu_m = ROOT.TCanvas()
h_pnu_m.Draw()
c_pnu_m.Draw()

In [28]:
c_CosTheta_W_minus = ROOT.TCanvas()
h_CosTheta_W_minus.SetMinimum(0)
h_CosTheta_W_minus.Draw()
c_CosTheta_W_minus.Draw()

c_l_minus_cosTheta_prime = ROOT.TCanvas()
h_l_minus_cosTheta_prime.SetMinimum(0)
h_l_minus_cosTheta_prime.Draw()
c_l_minus_cosTheta_prime.Draw()

c_l_minus_phi_prime = ROOT.TCanvas()
h_l_minus_phi_prime.SetMinimum(0)
h_l_minus_phi_prime.Draw()
c_l_minus_phi_prime.Draw()

c_l_plus_cosTheta_prime = ROOT.TCanvas()
h_l_plus_cosTheta_prime.SetMinimum(0)
h_l_plus_cosTheta_prime.Draw()
c_l_plus_cosTheta_prime.Draw()

c_l_plus_phi_prime = ROOT.TCanvas()
h_l_plus_phi_prime.SetMinimum(0)
h_l_plus_phi_prime.Draw()
c_l_plus_phi_prime.Draw()